In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def identity_transpose(A):
    '''Calculate (I - A^T)'''
    return tf.eye(A.shape[0], A.shape[0]) - tf.transpose(A)

def identity_transpose_inverse(A):
    '''Calculate (I - A^T)^(-1)'''
    return tf.linalg.inv(identity_transpose(A))

In [3]:
class Encoder(tf.keras.Model):
    '''
    Encoder class for DAG-GNN method

    Inputs:
    adjA (tensor [d, d]) : current estimated adjascency matrix
    ind_dim (int) : dimension of input layer
    hid_dim (int) : dimension of hidden layer
    out_dim (int) : dimension of output layer

    Outputs:
    out (tensor [batch, d]) : output of neural network
    ligs (tensor [d, d]) : product of (I - A^T @ out)
    adjA (tensor [d, d]) : current estimated adjascency matrix

    '''
    def __init__(self, adjA, in_dim, hid_dim, out_dim):
        super(Encoder, self).__init__()
        self.adjA = tf.Variable(initial_value = adjA, trainable = True)
        #self.Wa = tf.Variable(np.zeros(), trainable = True)

        self.fc1 = tf.keras.layers.Dense(hid_dim, activation= 'relu')
        self.fc2 = tf.keras.layers.Dense(out_dim)

    def call(self, inputs):
        '''Forward process of neural network'''
        #calculate I - A^T
        I_adjA = identity_transpose(self.adjA)
        hidden = self.fc1(inputs)
        outputs = self.fc2(hidden)
        logits = tf.matmul(I_adjA, outputs)
        return outputs, logits, self.adjA


class Decoder(tf.keras.Model):
    '''
    Decoder class for DAG-GNN method

    Inputs:
    ind_dim (int) : dimension of input layer
    out_dim (int) : dimension of output layer
    hid_dim (int) : dimension of hidden layer

    Outputs:
    '''
    def __init__(self, in_dim, hid_dim, out_dim):
        super(Decoder, self).__init__()
        self.fc1 = tf.keras.layers.Dense(hid_dim, activation = 'relu')
        self.fc2 = tf.keras.layers.Dense(out_dim)

    def call(self, z_inputs,  adjA):

        #calculate (I - A^T)^(-1)
        I_adjA = identity_transpose(adjA)
        z = tf.matmul(I_adjA, z_inputs)

        hidden = self.fc1(z)
        outputs = self.fc2(hidden)
        return z, outputs

In [4]:
class DAG_GNN_VAE(tf.keras.Model):
    '''
    Model class for DAG-GNN method training

    Inputs:
    adjA (tensor [d, d]) : current estimated adjascency matrix
    ind_dim (int) : dimension of input layer
    hid_dim (int) : dimension of hidden layer
    out_dim (int) : dimension of output layer

    Outputs:
    out (tensor [batch, d]) : output of neural network
    ligs (tensor [d, d]) : product of (I - A^T @ out)
    adjA (tensor [d, d]) : current estimated adjascency matrix

    '''
    def __init__(self, adjA, in_dim, hid_dim, out_dim):
        super(DAG_GNN_VAE, self).__init__()
        self.encoder = Encoder(adjA, in_dim, hid_dim, out_dim)
        self.decoder = Decoder(in_dim, hid_dim, out_dim)
    
    def call(self, inputs):
        en_outputs, logits, new_adjA = self.encoder(inputs)
        z, de_outputs = self.decoder(logits, new_adjA)
        return en_outputs, logits, new_adjA, z, de_outputs
        
    def _h(A):
        '''Calculate the constraint of A ensure that it's a DAG'''
        #(Yu et al. 2019 DAG-GNN)
        # h(w) = tr[(I + kA*A)^n_variables] - n_variables
        M = tf.eye(n_variables, num_columns = n_variables) + A/n_variables
        E = M
        for _ in range(n_variables - 2):
            E = tf.linalg.matmul(E, M)
        h = tf.math.reduce_sum(tf.transpose(E) * M) - n_variables
        return h
    
    def _loss(A, logits, X, Y):
        '''
        Function that evaluate the model loss
        loss = kl loss + nll loss + dag constraint + l1 reg + l2 reg
        '''
        # h constraint loss
        h = self._h(A)
        h_loss = 0.5 * rho * h * h + alpha * h
        
        #KL divergence
        kl_loss = tf.sum(tf.pow(logits, 2) / ( 2 * logits.shape[0]))
        
        #negative likelihood loss
        nll_loss = tf.sum(tf.pow(X - Y, 2) / (2 * n_variables))
        
        #L1 penalization
        l1_loss = lambda1 * tf.sum(tf.abs(A))
        
        #diagonal penalization
        diag_loss = 100 * tf.linalg.trace(A * A)
        
        loss = h_loss + kl_loss + nll_loss + l1_loss + diag_loss
        return loss
        

In [8]:
A = np.ones((5, 5)).astype(np.float32)
data = np.ones((5, 200)).astype(np.float32)
model = DAG_GNN_VAE(A, 5, 20, 5)
model.build(data.shape)

In [ ]:
model.fit()

In [ ]:
model.trainable_variables

In [ ]:
model(data)

In [ ]:
def dag_gnn(data, hid_dim = 20, h_tol = 1e-8, threshold = 0.3, lambda1 = 0.1, rho_max = 10e20, max_iter = 10e8, n_epochs = 20):
    '''
    Function that apply the DAG GNN method to estimate a DAG
    
    Inputs:
        data (numpy.matrix) : [n_samples, n_variables] samples matrix 
        hid_dim  (int) : list of dimensions for neural network hidden layer
        h_tol (float) : tolerance for constraint, exit condition 
        threshold (float) : threshold for W_est edge values
        lambda1 (float) : L1 regularization parameter
        rho_max (float) : max value for rho in augmented lagrangian
        max_iter (int) : max number of iterations
        n_epochs (int) : number of epochs
    Outputs:
        A_est (numpy.matrix): [n_variables, n_variables] estimated graph
    '''
    
    def update_optmizer(optimizer, rho):
        '''related LR to rho, whenever rho gets big, reduce LR proportionally'''
        MAX_LR = 1e-2
        MIN_LR = 1e-4
        
        new_lr = / (np.log10(rho) + 1e-10)
        lr = min(MAX_LR, max(MIN_LR, new_lr)) #if new_lr is inside limits, use it
        
        #update LR
        
        return optmizer, lr
    
    def train(optimizer):
        '''Model training'''
        #update optmizer
        
        optimizer, lr = update_optmizer(optimizer, rho)
        
        for epoch in range(n_epochs):
            for batch_id, batch_data in enumerate(train_loader):
                
                #passing through neural network
                en_outputs, logits, adjA, z, de_outputs = vae(batch_data)
                with tf.GradientTape() as tape:
                    tape.watch(vae.trainable_variables)
                    #calculate loss
                    loss = vae._loss(adjA, logits, decoder_out, batch_data)
                
                
                
                
        return adjA
    
    
    ########################
    # Optimization process #
    ########################
        
    n_variables = data.shape[1]
    rho, alpha, h = 1., 0., np.Inf
    
    train_loader, test_loader = setup_data_loader(data)
    
    #setup of neural networks
    new_adj = np.zeros((n_variables, n_variables))
    vae = DAG_GNN_VAE(new_adj, n_variables, hid_dim, n_variables)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    
    for _ in range(int(max_iter)):
        h_new = None
        while rho < rho_max:
            A_est = train() 
            h_new = _h(A_est)
                
            #Update constraint parameter rho
            if h_new > 0.25 * h:
                rho = rho*10
            else:
                break
        
        #Ascent alpha
        h = h_new    
        alpha += rho * h
        
        #Verifyng constraint tolerance
        if h <= h_tol or rho >= rho_max:
            break
    
    #Applyng threshold
    A_est[A_est < A_threshold] = 0
    return A_est
        

In [6]:
np.log10(10)

1.0